In [1]:
import pandas as pd
import numpy as np
import pyreadr
import math
import random
import os
import csv

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import os
import time

In [2]:
%%time 

# Abriendo los archivos .csv
folderpath =  r"C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data"
print("Files and directories in a specified path:")
files = []
filenames = []
for filename in os.listdir(folderpath):
    f = os.path.join(folderpath, filename)
    filenames.append(filename)
    files.append(f)
    if os.path.isfile(f):
        print(f)

Files and directories in a specified path:
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_programas_sociales_hogar.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_RE223132.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_RE516171.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_RE758081.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_REC84DV.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_REC91.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_REC93DVdisciplina.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_REC95.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_RECH23.csv
Wall time: 2.99 ms


In [3]:
%%time 

for i in range(len(filenames)):
    filenames[i] = filenames[i][slice(0,-4)]
filenames

Wall time: 0 ns


['endes_2021_programas_sociales_hogar',
 'endes_2021_RE223132',
 'endes_2021_RE516171',
 'endes_2021_RE758081',
 'endes_2021_REC84DV',
 'endes_2021_REC91',
 'endes_2021_REC93DVdisciplina',
 'endes_2021_REC95',
 'endes_2021_RECH23']

In [17]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()  # Launches a scheduler and workers locally
client = Client(cluster)  # Connect to distributed cluster and override default
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:59677/status,
Dashboard: http://127.0.0.1:59677/status,Workers: 4
Total threads: 8,Total memory: 7.81 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59678,Workers: 4
Dashboard: http://127.0.0.1:59677/status,Total threads: 8
Started: Just now,Total memory: 7.81 GiB
Comm: tcp://127.0.0.1:59693,Total threads: 2
Dashboard: http://127.0.0.1:59694/status,Memory: 1.95 GiB
Nanny: tcp://127.0.0.1:59683,


In [14]:
from distributed import Client
client = Client(cluster)  # some Dask args for configuration
import modin.pandas as pd  # Modin will connect to that client

In [4]:
import dask.dataframe as dd

In [5]:
#asignamos nombres a los dataframes
programas_sociales = dd.read_csv(files[0], dtype = str)
nacimiento = dd.read_csv(files[1], dtype = str)
nupcialidad_fecundidad = dd.read_csv(files[2], dtype = str)
sida_condon = dd.read_csv(files[3], dtype = str)
mortalidad_violencia = dd.read_csv(files[4], dtype = str)
MEF = dd.read_csv(files[5], dtype = str) 
disciplina_infantil = dd.read_csv(files[6], dtype = str)
inmunizacion_salud = dd.read_csv(files[7], dtype = str)
vivienda = dd.read_csv(files[8], dtype = str)

In [6]:
%%time 

# identificar bases que tengan CASEID
df_list = [programas_sociales, nacimiento, nupcialidad_fecundidad, sida_condon, mortalidad_violencia, MEF, disciplina_infantil,
          inmunizacion_salud,vivienda]

df_list_names = ["programas_sociales", "nacimiento", "nupcialidad_fecundidad", "sida_condon", "mortalidad_violencia", "MEF", "disciplina_infantil",
          "inmunizacion_salud","vivienda"]

df_CASEID = []

for df, df_name in zip(df_list,df_list_names):
       
    if 'CASEID' in df.columns:
        print(df_name)
        
        df_CASEID.append(df)

nacimiento
nupcialidad_fecundidad
sida_condon
mortalidad_violencia
MEF
disciplina_infantil
inmunizacion_salud
Wall time: 997 µs


In [7]:
%%time 

base_total = nacimiento.merge(nupcialidad_fecundidad, on = ['CASEID', 'ID1'], how ='left').merge(sida_condon, on = ['CASEID', 'ID1'], how ='left').merge(mortalidad_violencia, on = ['CASEID', 'ID1'], how ='left').merge(MEF, on = ['CASEID', 'ID1'], how ='left').merge(disciplina_infantil, on = ['CASEID', 'ID1'], how ='left').merge(inmunizacion_salud, on = ['CASEID', 'ID1'], how ='left')

Wall time: 334 ms


In [8]:
base_total.compute()

,ID1,CASEID,V201,V202,V203,V204,V205,V206,V207,V208,...,S466DC11,S466MC11,S466YC11,S466WC11,S466HC11,S466DC12,S466MC12,S466YC12,S466WC12,S466HC12
0,2021,000100301 2,1,1,0,0,0,0,0,1,...,,,,,,,,,,
1,2021,000100301 4,1,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,000100401 1,6,4,0,0,2,0,0,1,...,10,3,2020,8640,725,26,5,2020,9010,750
3,2021,000100801 2,4,1,3,0,0,0,0,1,...,,,,,,,,,,
4,2021,000100801 4,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43132,2021,325410601 1,7,4,3,0,0,0,0,2,...,,,,,,,,,,
43133,2021,325410601 1,7,4,3,0,0,0,0,2,...,21,7,2019,9950,790,29,10,2019,11300,860
43134,2021,325410601 2,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43135,2021,325410801 2,2,1,1,0,0,0,0,1,...,,,,,,,,,,


In [9]:
%%time 

df_final = base_total.compute()

Wall time: 4.24 s


In [10]:
df_final.dtypes

ID1         object
CASEID      object
V201        object
V202        object
V203        object
             ...  
S466DC12    object
S466MC12    object
S466YC12    object
S466WC12    object
S466HC12    object
Length: 1272, dtype: object

In [11]:
%%time 

# Seleccionando las variables de interés 
variables = ['ID1', 'SREGION', 'SPROVIN', 'SDISTRI', 'CASEID',
             'S108N', 'S119', 'S119D', 'V218', 'V213', 'V225',    # carac. personales
             'S45D4', 'S45D5', 'S45P4', 'S45P5', 'S45B0',    # vacunas
             'S45B1', 'S45B2', 'S45B3', 'S45F1', 'S45F2',    # vacunas
             'V501', 'V714', 'D104', 'D106', 'D107', 'D108']

data_final = df_final[variables]
data_final.shape

Wall time: 399 ms


(43137, 27)

In [12]:
for i in variables:
        if data_final[i].dtype == 'O':
            data_final[i] = data_final[i].str.replace(" ", "")
            
            # Reemplazando los missing values por NaN
            data_final[i] = data_final[i].replace(' ', np.nan)

            # Convirtiendo los datos a valores numéricos
            data_final[i] =  np.floor(pd.to_numeric(data_final[i], errors='coerce'))
            
            # Convirtiendo los datos a valores numéricos
            data_final[i] =  data_final[i].astype('Int64')

In [13]:
data_final.dtypes

ID1        Int64
SREGION    Int64
SPROVIN    Int64
SDISTRI    Int64
CASEID     Int64
S108N      Int64
S119       Int64
S119D      Int64
V218       Int64
V213       Int64
V225       Int64
S45D4      Int64
S45D5      Int64
S45P4      Int64
S45P5      Int64
S45B0      Int64
S45B1      Int64
S45B2      Int64
S45B3      Int64
S45F1      Int64
S45F2      Int64
V501       Int64
V714       Int64
D104       Int64
D106       Int64
D107       Int64
D108       Int64
dtype: object

In [14]:
data_final = data_final.fillna(0)

In [15]:
data_final.dtypes

ID1        Int64
SREGION    Int64
SPROVIN    Int64
SDISTRI    Int64
CASEID     Int64
S108N      Int64
S119       Int64
S119D      Int64
V218       Int64
V213       Int64
V225       Int64
S45D4      Int64
S45D5      Int64
S45P4      Int64
S45P5      Int64
S45B0      Int64
S45B1      Int64
S45B2      Int64
S45B3      Int64
S45F1      Int64
S45F2      Int64
V501       Int64
V714       Int64
D104       Int64
D106       Int64
D107       Int64
D108       Int64
dtype: object

In [16]:
# Agregando los datos para evaluar cuántas mujeres han sido víctimas de violencia en al menos una de sus modalidades
data_final['violencia'] = data_final['D104'] + data_final['D106'] + data_final['D107'] + data_final['D108']

# Contabilizando lo encontrado
data_final['violencia'].value_counts()

0    33729
1     4372
2     2621
3     1598
4      817
Name: violencia, dtype: Int64

In [17]:
# Creando variable dummy 

data_final['y'] = data_final['violencia']
data_final.loc[(data_final['y'] > 0), 'y'] = 1
data_final['y'].value_counts()

0    33729
1     9408
Name: y, dtype: Int64

In [18]:
from sklearn.linear_model import *
from sklearn.pipeline import *
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import *
import pandas as pd

Secuencial

In [19]:
data_final.columns

Index(['ID1', 'SREGION', 'SPROVIN', 'SDISTRI', 'CASEID', 'S108N', 'S119',
       'S119D', 'V218', 'V213', 'V225', 'S45D4', 'S45D5', 'S45P4', 'S45P5',
       'S45B0', 'S45B1', 'S45B2', 'S45B3', 'S45F1', 'S45F2', 'V501', 'V714',
       'D104', 'D106', 'D107', 'D108', 'violencia', 'y'],
      dtype='object')

In [20]:
cols = ['y', 'SREGION', 'SPROVIN', 'SDISTRI', 'S108N', 'S119',
       'S119D', 'V218', 'V213', 'V225', 'S45D4', 'S45D5', 'S45P4', 'S45P5',
       'S45B0', 'S45B1', 'S45B2', 'S45B3', 'S45F1', 'S45F2', 'V501', 'V714']

In [21]:
data_final = data_final[cols]

In [22]:
data_final = data_final[data_final['y'].notna()]

In [23]:
data_final = data_final.fillna(0)

In [34]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [35]:
data_final  = clean_dataset(data_final)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(
    data_final.iloc[:,range(1,len(data_final.columns))],
    data_final['y'],
    test_size=0.3,
    random_state=175904)
print("Number of records: %d"%data_final.shape[0])
data_final[1:5]

Number of records: 43137


,y,SREGION,SPROVIN,SDISTRI,S108N,S119,S119D,V218,V213,V225,...,S45P4,S45P5,S45B0,S45B1,S45B2,S45B3,S45F1,S45F2,V501,V714
1,0.0,3.0,1.0,1.0,3.0,10.0,6.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
2,1.0,3.0,1.0,1.0,0.0,10.0,7.0,6.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0
3,0.0,3.0,1.0,1.0,3.0,10.0,5.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
4,0.0,3.0,1.0,1.0,2.0,10.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
%%time 
grid_params = { 
    "lasso0__eps": [1e-3, 1e-4, 1e-5],
    "lasso0__n_alphas": [100, 250, 500]}
lasso0 = LassoCV(max_iter=1000, n_alphas=300, n_jobs=15, random_state=175904)
pipe0  = Pipeline([("lasso0", lasso0)])
grid = GridSearchCV(estimator=pipe0, 
                    param_grid=grid_params, 
                    n_jobs=14, scoring="neg_mean_squared_error")
gridm = grid.fit(X_train, y_train)

Wall time: 7.57 s


In [38]:
print("Score: %f with metric '%s'\nThe best parameters are: %s"%(
    gridm.score(X_test, y_test), 
    grid.scoring,
    gridm.best_params_))

Score: -0.145515 with metric 'neg_mean_squared_error'
The best parameters are: {'lasso0__eps': 1e-05, 'lasso0__n_alphas': 500}


In [39]:
%%time
gridm.predict(X_test)

Wall time: 1.99 ms


array([-0.00816288,  0.15144114,  0.3848665 , ...,  0.26314418,
        0.17988786,  0.35681079])

Pararelo

In [28]:
import dask.dataframe as dd
import dask.array as da
import dask_ml.model_selection as dms
from dask.distributed import Client
import dask
import numpy as np
from dask import delayed

In [40]:
dX_train, dX_test, dy_train, dy_test = (
    dd.from_pandas(X_train, npartitions=3),
    dd.from_pandas(X_test, npartitions=3),
    dd.from_pandas(y_train, npartitions=3),
    dd.from_pandas(y_test, npartitions=3)
)

data_final.head(4)

,y,SREGION,SPROVIN,SDISTRI,S108N,S119,S119D,V218,V213,V225,...,S45P4,S45P5,S45B0,S45B1,S45B2,S45B3,S45F1,S45F2,V501,V714
0,0.0,3.0,1.0,1.0,3.0,10.0,5.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
1,0.0,3.0,1.0,1.0,3.0,10.0,6.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
2,1.0,3.0,1.0,1.0,0.0,10.0,7.0,6.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0
3,0.0,3.0,1.0,1.0,3.0,10.0,5.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0


In [41]:
%%time 
grid_params = { 
    "lasso0__eps": [1e-3, 1e-4, 1e-5],
    "lasso0__n_alphas": [100, 250, 500]}

lasso0 = LassoCV(max_iter=1000, n_alphas=300, n_jobs=15, random_state=175904)
pipe0  = Pipeline([("lasso0", lasso0)])
dgrid = dms.GridSearchCV(estimator=pipe0, 
                    param_grid=grid_params, 
                    n_jobs=14, scoring="neg_mean_squared_error")
dgridm = dgrid.fit(dX_train, dy_train)

Wall time: 11.1 s


In [42]:
%%time
y_hat = dX_test.map_partitions(dgridm.predict)
y_hat.compute()

Wall time: 578 ms


array([0.22190122, 0.42825253, 0.62131866, ..., 0.36036112, 0.01290938,
       0.34446537])